<a href="https://colab.research.google.com/github/ameyaoka/ml-notebooks/blob/main/movie_genre_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Movie_genre_classification




In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 38.0 MB/s eta 0:00:00


In [ ]:
  pip install --upgrade huggingface_hub

In [ ]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd

import torch

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

# If the dataset is gated/private, make sure you have run huggingface-cli login
dataset = load_dataset("datadrivenscience/movie-genre-prediction")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/54000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/36000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/datadrivenscience___parquet/datadrivenscience--movie-genre-prediction-01acd85570f2b187/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'genre'],
        num_rows: 54000
    })
    test: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'genre'],
        num_rows: 36000
    })
})

In [ ]:
dataset = dataset.class_encode_column("genre")

Casting to class labels:   0%|          | 0/54000 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/36000 [00:00<?, ? examples/s]

In [ ]:
train_data=dataset['train']
train_data

Dataset({
    features: ['id', 'movie_name', 'synopsis', 'genre'],
    num_rows: 54000
})

In [ ]:
test_data = dataset['test']

In [ ]:
validation= train_data.train_test_split(test_size=0.1, stratify_by_column="genre")

In [ ]:
validation

DatasetDict({
    train: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'genre'],
        num_rows: 48600
    })
    test: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'genre'],
        num_rows: 5400
    })
})

In [ ]:
from datasets import DatasetDict

In [ ]:
new_dataset = DatasetDict({
    'train': validation['train'],
    'test': test_data,
    'valid': validation['test']})

In [ ]:
new_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'genre'],
        num_rows: 48600
    })
    test: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'genre'],
        num_rows: 36000
    })
    valid: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'genre'],
        num_rows: 5400
    })
})

- Dataset consist of two sets 1. train  2. test

- movie id is not required for training .
- movie_name is our first input and synopsis is our second input
- genre is out label.


- The dataset is balanced

## tokenisation


1.     Find a list of all unique words in the dataset.

2.    Associate each word/token with a unique number. This is called vocab.

3.  Encode the dataset using the mapping present in the vocab.


- Model max length defines the maximum number of tokens that a single data sample can have, i.e. in the above case, our model DistilBert can accept text sequences of up to 512 tokens long.

- Model input names are the fields that the model will take as inputs for training and inference purposes.

In [ ]:
checkpoint = "bert-base-uncased"

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

- token_type_id tells model which part of input is first sentence and which part is second sentence.

In [ ]:
def tokenize_function(example):
    return tokenizer(example["movie_name"], example["synopsis"], truncation=True, padding =True ,return_tensors="pt")



In [ ]:
tokenized_dataset = new_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/48600 [00:00<?, ? examples/s]

Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5400 [00:00<?, ? examples/s]

- **Our tokenized dataset consist of id , movie_name , synopsis genre input_id token_type_id , attnetion mask .**
- **we need attetion_mask , inputids , labels , tokent_type_ids**


In [ ]:
tokenized_dataset['train'][0]

{'id': 897,
 'movie_name': 'The Singer and the Bouncer',
 'synopsis': 'Only Her Sweet and Gentle Songs Can Melt The Tough Heart Of a Hardened Men In A Cruel World.',
 'genre': 0,
 'input_ids': [101,
  1996,
  3220,
  1998,
  1996,
  17523,
  2099,
  102,
  2069,
  2014,
  4086,
  1998,
  7132,
  2774,
  2064,
  14899,
  1996,
  7823,
  2540,
  1997,
  1037,
  15015,
  2273,
  1999,
  1037,
  10311,
  2088,
  1012,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [ ]:

tokenized_dataset = tokenized_dataset.rename_column('genre',"labels")

tokenized_dataset.set_format('torch')


In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 48600
    })
    test: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 36000
    })
    valid: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5400
    })
})

In [ ]:
tokenized_dataset  =tokenized_dataset.remove_columns(['id','movie_name','synopsis'])

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 48600
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 36000
    })
    valid: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5400
    })
})

- The labels are string so we need to use huggingface ClassLabel

In [ ]:
tokenized_dataset["train"][5]

{'labels': tensor(6),
 'input_ids': tensor([  101,  8463,  2000, 22947,   102,  9106,  2003,  8733,  2000,  8620,
          1996,  8215,  2000,  1037,  2647,  6106,  9689,  2067,  2000,  2010,
         10759,  2044,  2010,  2269,  1005,  1055, 10102,  1012,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
from transformers import  DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:

from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_dataset["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)

eval_dataloader = DataLoader(
    tokenized_dataset["valid"], batch_size=8, collate_fn=data_collator
)

test_dataloader = DataLoader(
    tokenized_dataset["test"], batch_size=8, collate_fn=data_collator
)


In [ ]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 85]),
 'token_type_ids': torch.Size([8, 85]),
 'attention_mask': torch.Size([8, 85])}

 - label2id needs testing

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=10)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

**To make sure that everything will go smoothly during training, we pass our batch to this model:**
- **if your model outputs 4 classes, CrossEntropyLoss expects your class labels to be in the range [0,C−1]**

In [ ]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(2.2518, grad_fn=<NllLossBackward0>) torch.Size([8, 10])


- **All 🤗 Transformers models will return the loss when labels are provided, and we also get the logits (two for each input in our batch, so a tensor of size 8 x 2).**

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

18225


In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/18225 [00:00<?, ?it/s]

In [ ]:
model.push_to_hub('Ammok/movie_genre_2',create_pr=1)

CommitInfo(commit_url='https://huggingface.co/Ammok/movie_genre_2/commit/381e31718714d0e93534b03aa47f878a3b8b687e', commit_message='Upload BertForSequenceClassification', commit_description='', oid='381e31718714d0e93534b03aa47f878a3b8b687e', pr_url='https://huggingface.co/Ammok/movie_genre_2/discussions/1', pr_revision='refs/pr/1', pr_num=1)

In [ ]:
tokenizer.push_to_hub('Ammok/movie_genre_2',create_pr=1)

CommitInfo(commit_url='https://huggingface.co/Ammok/movie_genre_2/commit/0e601d95f5ecb54d2a8ccfc5f802079109f8504a', commit_message='Upload tokenizer', commit_description='', oid='0e601d95f5ecb54d2a8ccfc5f802079109f8504a', pr_url='https://huggingface.co/Ammok/movie_genre_2/discussions/2', pr_revision='refs/pr/2', pr_num=2)

In [ ]:
model.save_pretrained("/home")
tokenizer.save('/home')

AttributeError: ignored

In [ ]:
pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.1 MB/s eta 0:00:00


In [ ]:
metrics = evaluate.combine([
    evaluate.load("precision", average="weighted"),
    evaluate.load("recall", average="weighted")
])

In [ ]:
import evaluate
from datasets import load_metric
metric = load_metric('recall')

model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

<ipython-input-52-907c8b8135ce>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('recall')


ValueError: ignored

**loading finetuned model using transformers library**




In [ ]:
text1= "The Syndicate"
text2 = "An alcoholic, his wife, a down and out American pilot and a young German form a prospecting team to search for uranium in Africa and find themselves plagued by sinister accidents."

In [ ]:
tokenT=tokenizer(text1,text2,return_tensors='pt')

In [ ]:
tokenT

{'input_ids': tensor([[  101,  1996, 16229,   102,  2019, 14813,  1010,  2010,  2564,  1010,
          1037,  2091,  1998,  2041,  2137,  4405,  1998,  1037,  2402,  2446,
          2433,  1037,  9824,  2075,  2136,  2000,  3945,  2005, 14247,  1999,
          3088,  1998,  2424,  3209, 17808,  2011, 16491, 13436,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
labels = ['fantasy', 'horror', 'family', 'scifi', 'action', 'crime','adventure', 'mystery', 'romance', 'thriller']

In [ ]:
id2label = {idx: label for idx, label in enumerate(labels)}
label2id = {label: idx for idx, label in enumerate(labels)}


In [ ]:
tokenizer.save_pretrained("/home")

('/home/tokenizer_config.json',
 '/home/special_tokens_map.json',
 '/home/vocab.txt',
 '/home/added_tokens.json',
 '/home/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification,AutoConfig

tokenizer = AutoTokenizer.from_pretrained("Ammok/movie_genre_2")
config = AutoConfig.from_pretrained("Ammok/movie_genre_2", label2id=label2id, id2label=id2label)
model_ftuned = AutoModelForSequenceClassification.from_pretrained("Ammok/movie_genre_2",config=config)

In [ ]:
test = tokenizer(text1,text2,return_tensors='pt')
test

NameError: ignored

In [ ]:

with torch.no_grad():
    logits = model_ftuned(**test).logits



NameError: ignored

In [ ]:


predicted_class_id = logits.argmax().item()
model_ftuned.config.id2label[predicted_class_id]





'thriller'

In [ ]:
test_token

NameError: ignored

In [ ]:
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm
pipe = pipeline("sentiment-analysis", tokenizer = tokenizer ,model=model_ftuned, config=config,device=0)
test_token = new_dataset['test']
submission = []
# KeyDataset (only *pt*) will simply return the item in the dict returned by the dataset item
# as we're not interested in the *target* part of the dataset. For sentence pair use KeyPairDataset
for out in pipe(KeyDataset(test_token,"synopsis"), batch_size=8):
      submission.append(out)
    # {"text": "NUMBER TEN FRESH NELLY IS WAITiING ON YOU GOOD NIGHT HUSBAND"}
    # {"text": ....}
    # ....

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
submission[:8]

[{'label': 'scifi', 'score': 0.9641838669776917},
 {'label': 'crime', 'score': 0.6841937899589539},
 {'label': 'action', 'score': 0.42166027426719666},
 {'label': 'adventure', 'score': 0.7252926230430603},
 {'label': 'action', 'score': 0.349602609872818},
 {'label': 'romance', 'score': 0.409827321767807},
 {'label': 'adventure', 'score': 0.3636525273323059},
 {'label': 'family', 'score': 0.449344664812088}]

In [ ]:
res = [item['label'] for item in submission]

In [ ]:

res[0]

'scifi'

In [ ]:
ids = dataset ['test']['id']

In [ ]:
ids[:8]

[16863, 48456, 41383, 84007, 40269, 16524, 21245, 84681]

In [ ]:
submit = pd.DataFrame(
    {'ids': ids,
     'genre':res,
      })

In [ ]:
submit

,ids,genre
0,16863,scifi
1,48456,crime
2,41383,action
3,84007,adventure
4,40269,action
...,...,...
35995,73156,horror
35996,21242,family
35997,70135,mystery
35998,85987,thriller


In [ ]:
submit.to_csv('submission.csv')